In [16]:
import numpy as np
import pandas as pd
from gensim import corpora
import gensim.models as gsm
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec

In [163]:
t= pd.read_table('output_rj.txt', names=['docid', 'pageid','text'], header=None, delimiter="\t", quoting=3,encoding = "utf-8")

In [164]:
t

,docid,pageid,text
0,12,0,Anarchism is often defined as a political phil...
1,12,1,"However, others argue that while anti-statism ..."
2,12,2,"Therefore, they argue instead that anarchism e..."
3,12,3,"Proponents of anarchism, known as ""anarchists""..."
4,12,4,"As a subtle and anti-dogmatic philosophy, anar..."
5,12,5,Anarchism does not offer a fixed body of doctr...
6,12,6,There are many types and traditions of anarchi...
7,12,7,Anarchist schools of thought can differ fundam...
8,12,8,Strains of anarchism have often been divided i...
9,12,9,Anarchism is often considered a radical left-w...


In [165]:
t['text'][0]

'Anarchism is often defined as a political philosophy which holds the state to be undesirable, unnecessary, or harmful.'

In [61]:
def text_to_words( text ):
    #convert emoji into name and append at the end of text
    text = str(text)
    for char in text:
        if char in emoji.UNICODE_EMOJI:
            text = text + unicodedata.name(char) + " "

    #remove any url to URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',text)
    #Convert any @Username to "AT_USER"
    text = re.sub('@[^\s]+','AT_USER',text)
    
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    text = re.sub('[\n]+', ' ', text)
    #Remove not alphanumeric symbols white spaces
    text = re.sub(r'[^\w]', ' ', text)
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')
    
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    tokens = word_tokenize(letters_only.lower())
    filtered = [word for word in tokens if word not in stop_words]
    stemmed = [stemmer.stem(word) for word in filtered] 
    
    return  stemmed

In [166]:
size = len(t['text'])
text_clean = []
for i in range(size):
    #print states after 10000 reviews
    if ((i+1)%10000 == 0):
        print("Review %d of %d\n" % ( i+1, size ) )
    text_clean.append( text_to_words( t['text'][i] ) )

Review 10000 of 14966



In [17]:
model = gsm.KeyedVectors.load_word2vec_format('word2vec_twitter_model.bin', binary=True,unicode_errors='ignore')

In [18]:
def makeFeatureVec_w2v(words):

    featureVec = np.zeros((400)).astype(float)  
    nwords = 0
    
    for word in words:
        if word in model.vocab:
            embedding_vector = model[word]
            if embedding_vector is not None: 
                nwords = nwords + 1
                featureVec = np.add(featureVec, embedding_vector)
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0:
        featureVec = np.divide(featureVec,nwords)
    return featureVec

In [169]:
text_vector = []
size = len(t['text'])
for i in range(size):
    docid = np.asarray(t['docid'][i]).astype(float).reshape((1))
    pageid = np.asarray(t['pageid'][i]).astype(float).reshape((1))
    temp = np.concatenate((docid, pageid))
    text_vector.append(np.concatenate((temp, makeFeatureVec_w2v(text_clean[i]))))

In [173]:
text_vector = np.asarray(text_vector)
np.savetxt('output_vector.txt', text_vector, delimiter=",", fmt="%.10f") 